<a href="https://colab.research.google.com/github/SarthakJha-IITMandi/DS3-2021/blob/main/LSTM_IITMandi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses as lo_ss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import string
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.utils import class_weight
from sklearn.metrics import classification_report , roc_auc_score
import keras
import keras.backend as K
from google.colab import drive
from numpy import savetxt
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
from sklearn.datasets import make_classification
from sklearn.utils import class_weight
from sklearn.metrics import classification_report , roc_auc_score

## Data preprocessing

### Reading data and standardizing it

In [ ]:
# Reading data from csv file

import os
github_raw_link="https://raw.githubusercontent.com/Aryansh085/LAP/main/858417d1-7d54-4115-a01b-fdda5e03ada3_testing_combined_rows4035_disc_1_0p9_MULTIPLY_preproces_155cols.csv"
df1=pd.read_csv((github_raw_link),header = None)
df1=np.array(df1)

df1.shape

(4035, 155)

In [ ]:
# Making a new dataframe which contains only relevant features
df = df1[:,5:]
df = np.array(df)

# Standardizing data
from sklearn.preprocessing import StandardScaler
slr=StandardScaler()
slr.fit(df)
df = slr.transform(df)

# final dataframe
df=pd.DataFrame(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,1.750036,0.350342,-1.341973,-2.795068,0.432952,0.474938,-1.574554,-0.439485,-1.486932,-1.585869,...,-1.015021,-0.795066,-1.952407,-0.866407,-3.145759,3.099274,-0.818265,-0.095863,-0.652975,-0.693894
1,-0.254578,1.008169,0.819884,-1.666252,-0.495354,-0.246320,-0.306173,-0.182138,0.162858,-0.850303,...,1.485140,-0.294316,1.076433,-0.578114,0.625668,0.116021,1.182070,0.071192,1.467883,0.099011
2,-0.546815,-0.789877,-0.896693,0.609252,-1.785813,0.671458,-1.346309,0.051911,1.257504,0.635510,...,1.064428,0.506085,-0.692270,0.803917,0.736296,-0.473514,0.512632,-0.367337,-0.455865,-0.193491
3,1.129433,1.576726,-1.089738,-0.159556,-0.237411,-0.771867,1.524618,-0.068071,-0.171521,0.341681,...,-0.958503,-0.795708,-0.132790,0.018378,-0.432966,-0.066116,-0.006371,-0.472373,0.181778,0.074862
4,-0.692797,-0.435498,-1.673149,0.045856,0.292185,-1.561811,0.780544,-0.000231,-1.402506,0.866888,...,0.353773,-1.160216,-0.357106,0.220035,0.144812,-0.401812,0.585473,-0.644445,0.811905,-0.621054


### Manipulating data according to the timestamp

In [ ]:
# timestamp is defined here 
t = 5

In [ ]:
# Storing output corresponding to the new dataframe and according to the timestamp
y=np.array(df1[:,0])
y=y[t:]

y, y.shape

(array([2., 2., 2., ..., 0., 2., 2.]), (4030,))

In [ ]:
# Making 3d array where each row contains last t rows information of the data
x = np.array(df)
# print(x.shape,y.shape)

x_3d = []
for i in range(4035-t):
  tmp = []
  for j in range(t):
    tmp.append(x[j+i])
  x_3d.append(tmp)
x_3d = np.array(x_3d)
x_3d.shape

(4030, 5, 150)

In [ ]:
# making test and train input and output data for the lstm layer
total_size = x_3d.shape[0]

# assigning train-test ratio to split data in train-test
train_test_ratio=0.8
train_size = int(total_size * train_test_ratio)
test_size = total_size - train_size

In [ ]:
# these array will contain test and train data
x_train, y_train, x_test, y_test = [], [], [], []

# Random assignment of data : filling the test and train data randomly
for i in range(x_3d.shape[0]):
  feature = []
  corr_output = []

  no = np.random.randint(100)

  if(no < 80):
    feature = x_3d[i]
    corr_output = y[i]

    x_train.append(feature) 
    y_train.append(corr_output) 

  else:
    feature = x_3d[i]
    corr_output = y[i]

    x_test.append(feature) 
    y_test.append(corr_output) 

# changing each to np array
main_x_train = np.array(x_train)
main_y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(main_x_train.shape, main_y_train.shape)
print(x_test.shape, y_test.shape)

(3262, 5, 150) (3262,)
(768, 5, 150) (768,)


In [ ]:
# Sequentially assignment of data : Taking the first data as train data and last one as test data
x_train = x_3d[ : train_size]; x_test = x_3d[train_size : ]
y_train = y[ : train_size]; y_test = y[train_size : ]

# changing each to np array
main_x_train = np.array(x_train)
main_y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(main_x_train.shape, main_y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# count of values in different classes
class0_count = np.count_nonzero(main_y_train == 0)
class1_count = np.count_nonzero(main_y_train == 1)
class2_count = np.count_nonzero(main_y_train == 2)

print(class0_count, class1_count, class2_count)

### Handling Class imbalance

In [ ]:
# Random Over-sampling
x_train_1, y_train_1 = [], []
c1, c2, c3 = 0, 0, 0

for i in range(main_x_train.shape[0]):
  if(y_train[i] == 0):
    x_train_1.append(main_x_train[i])
    y_train_1.append(0)
    c1+=1
  elif(y_train[i] == 1):
    x_train_1.append(main_x_train[i])
    y_train_1.append(1)
    c2+=1
  elif(y_train[i] == 2 ):
    x_train_1.append(main_x_train[i])
    y_train_1.append(2)
    c3=c3+1

for i in range(main_x_train.shape[0]):
  if(y_train[i] == 1):
    no = np.random.randint(100)
    if(no >= 30):
      x_train_1.append(main_x_train[i])
      y_train_1.append(1)
      c2 += 1
  elif(y_train[i] == 2 ):
    no = np.random.randint(100)
    if(no >= 27):
      x_train_1.append(main_x_train[i])
      y_train_1.append(2)
      c3 += 1

x_train = x_train_1; y_train = y_train_1
x_train = np.array(x_train); y_train = np.array(y_train)

print(x_train.shape, y_train.shape)

c1, c2, c3

(4558, 5, 150) (4558,)


(1483, 1459, 1616)

In [ ]:
# Over-sampling with repeating just after
x_train_1, y_train_1 = [], []
c1, c2, c3 = 0, 0, 0

for i in range(main_x_train.shape[0]):
  if(y_train[i] == 0):
    x_train_1.append(main_x_train[i])
    y_train_1.append(0)
    c1 += 1
  elif(y_train[i] == 1):
    x_train_1.append(main_x_train[i]); x_train_1.append(main_x_train[i])
    y_train_1.append(1); y_train_1.append(1)
    c2 += 2
  elif(y_train[i] == 2 ):
    x_train_1.append(main_x_train[i]); x_train_1.append(main_x_train[i])
    y_train_1.append(2); y_train_1.append(2)
    c3 += 2

x_train = x_train_1; y_train = y_train_1
x_train = np.array(x_train); y_train = np.array(y_train)

print(x_train.shape, y_train.shape)

c1, c2, c3, c1 + c2 + c3

In [ ]:
# Random under-sampling 

print("main_x_train : ", main_x_train.shape)

x_train_1, y_train_1 = [], []
c1, c2, c3 = 0, 0, 0

for i in range(main_x_train.shape[0]):
  if(main_y_train[i] == 0):
    to_take = np.random.randint(100)
    if(to_take > 50):
      x_train_1.append(main_x_train[i])
      y_train_1.append(0)
      c1 += 1
  elif(main_y_train[i] == 1):
    x_train_1.append(main_x_train[i]) #; x_train_1.append(main_x_train[i])
    y_train_1.append(1) #; y_train_1.append(1)
    c2 += 1
  elif(main_y_train[i] == 2 ):
    x_train_1.append(main_x_train[i]) #; x_train_1.append(main_x_train[i])
    y_train_1.append(2) #; y_train_1.append(2)
    c3 += 1

x_train = x_train_1; y_train = y_train_1
x_train = np.array(x_train); y_train = np.array(y_train)

print(x_train.shape, y_train.shape)

c1, c2, c3, c1 + c2 + c3

In [ ]:
# Different weights for the classes to be assigned

sklearn_weights = class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(y_train),y = y_train)
sklearn_weights = dict(zip(np.unique(y_train),sklearn_weights))

# manually assigning weights
# sklearn_weights={0:1738,1:2348,2:2362}

sklearn_weights

{0: 1.0317129629629629, 1: 1.0433052434456929, 2: 0.9326218874241473}

## Models

### F1-score function

In [ ]:
import keras 

#import keras backend
import keras.backend as K


# f1-score function
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

### Models

In [ ]:
# Model 1

model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(t ,150,)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=1)
test_output = model.predict(x_test, verbose=1)
# history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)
# test_output = model.predict(x=, verbose=0)
# print(test_output)

### Function to return the model so that we can save it

In [ ]:
def create_model():
  model = Sequential()
  model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=( t,150,)))
  model.add(LSTM(400, activation='relu', return_sequences=True))
  model.add(LSTM(800, activation='relu', return_sequences=True))
  model.add(LSTM(150, activation='relu'))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(3,activation='sigmoid'))
  optimi=keras.optimizers.Adam()
  model.compile(optimizer=optimi, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
# with class weights
model=create_model()
# history = model.fit(x_train, y_train, epochs=30, validation_split=0.2, verbose=1)
history = model.fit(x_train, y_train, epochs=25, validation_split=0.2, verbose=1,class_weight=sklearn_weights)
model.save('model.h5')
test_output = model.predict(x_test, verbose=1)

Epoch 1/25
112/112 [==============================] - 15s 55ms/step - loss: 1.0861 - accuracy: 0.4025 - val_loss: 1.2423 - val_accuracy: 0.0000e+00
Epoch 2/25
112/112 [==============================] - 5s 48ms/step - loss: 1.0248 - accuracy: 0.4755 - val_loss: 1.0999 - val_accuracy: 0.3733
Epoch 3/25
112/112 [==============================] - 6s 49ms/step - loss: 0.8522 - accuracy: 0.5919 - val_loss: 1.0014 - val_accuracy: 0.4854
Epoch 4/25
112/112 [==============================] - 5s 49ms/step - loss: 0.6872 - accuracy: 0.6665 - val_loss: 0.7599 - val_accuracy: 0.5953
Epoch 5/25
112/112 [==============================] - 6s 52ms/step - loss: 0.5402 - accuracy: 0.7498 - val_loss: 0.5972 - val_accuracy: 0.7085
Epoch 6/25
112/112 [==============================] - 6s 56ms/step - loss: 0.3712 - accuracy: 0.8457 - val_loss: 0.4139 - val_accuracy: 0.8464
Epoch 7/25
112/112 [==============================] - 8s 72ms/step - loss: 0.2697 - accuracy: 0.9066 - val_loss: 0.2769 - val_accuracy: 0

In [ ]:
# Model 2
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(t, 150,)))
model.add(LSTM(400, activation='relu', return_sequences=True))
model.add(LSTM(800, activation='relu', return_sequences=True))
model.add(LSTM(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='relu'))
model.add(Dense(3,activation='sigmoid'))
optimi=keras.optimizers.Adam()

# added f1-score here
model.compile(optimizer=optimi, loss='sparse_categorical_crossentropy',metrics=['accuracy',get_f1])

# fitting model for different epochs
# without class weights
# history = model.fit(x_train, y_train, epochs=30, validation_split=0.2, verbose=1)

# # with class weights
model=create_model()
history = model.fit(x_train, y_train, epochs=30, validation_split=0.2, verbose=1)
test_output = model.predict(x_test, verbose=1)

In [ ]:
# Model 3
model3 = Sequential()
# model3.add(LSTM(800, activation='relu', return_sequences=True, input_shape=( t,150,)))
# model3.add(LSTM(400, activation='relu', return_sequences=True))
# model3.add(LSTM(200, activation='relu', return_sequences=True))
model3.add(LSTM(200, activation='relu', input_shape=( t,150,)))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(64, activation='relu'))

model3.add(Dense(10, activation='sigmoid'))
model3.add(Dense(3,activation='sigmoid'))
optimi=keras.optimizers.Adam()

# added f1-score here
model3.compile(optimizer=optimi, loss='sparse_categorical_crossentropy',metrics=['accuracy',get_f1])

# fitting model3 for different epochs
# history = model3.fit(x_train, y_train, epochs=30, validation_split=0.2, verbose=1)
history = model3.fit(x_train, y_train, epochs=20, validation_split=0.2, verbose=1, class_weight=sklearn_weights)
test_output3 = model3.predict(x_test, verbose=1)